# Load the data

In [109]:
import pandas as pd

#gen_pred_orig = pd.read_csv("../data/gen_predictions/predictions_bart-large-swipe_asset-test.csv", index_col = 0)
#gen_pred_ft = pd.read_csv("../data/gen_predictions/predictions_bart-large-swipe-adamw-paged8bit_asset-test.csv")
test_ds_llama_ft_alpaca = pd.read_csv("../data/gen_predictions/predictions_llama3.2-ft-alpaca-test_ds_c4spo.csv")
test_ds_llama_instruct = pd.read_csv("../data/gen_predictions/predictions_llama3.2-instruct-few-shot-test_ds_c4spo.csv")
#test_ds_c4sp_llamainstr = pd.read_csv("../data/gen_predictions/predictions_llama-instr-test_ds_c4sp-context_elab-seperately.csv")
test_ds_bart = pd.read_csv("../data/gen_predictions/predictions_bart-cs-subject.csv")
#test_ds_c4osp_llamainstr = pd.read_csv("../data/gen_predictions/predictions_llama-instr-test_ds_c4osp-context_elab-seperately.csv")

df_gen =test_ds_llama_instruct #test_ds_bart
#df_gen = df_gen.fillna("")
df_gen.isnull().sum()

Unnamed: 0              0
source_text             0
elaboration_sentence    0
pred_elaboration        0
dtype: int64

# Load datasets

In [2]:
from datasets import load_dataset

asset_dataset = load_dataset("facebook/asset", "simplification")
asset_dataset

DatasetDict({
    validation: Dataset({
        features: ['original', 'simplifications'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['original', 'simplifications'],
        num_rows: 359
    })
})

In [10]:
df_gen = gen_pred_ft
dataset = asset_dataset

# SARI - Hugging Face

In [35]:
from evaluate import load
from tqdm.notebook import tqdm
import pandas as pd

sari_metric = load("sari")
sari_scores = []

for index, row in tqdm(df_gen.iterrows(), total=len(df_gen)):
    r_content = row['text'] 
    s_content = dataset['test'][index] 
    prediction = row['prediction']  
    
    sari_score = sari_metric.compute(
        sources=[r_content],
        predictions=[prediction],
        references=[s_content['simplifications']]
    )
    
    sari_scores.append(sari_score['sari'])

  0%|          | 0/359 [00:00<?, ?it/s]

# SARI - EASSE package

In [13]:
from easse.sari import corpus_sari
from tqdm.notebook import tqdm


sari_scores_easse = []

for index, row in tqdm(df_gen.iterrows(), total=len(df_gen)):
    r_content = row['source_text'] 
    s_content = row['label_text']  
    prediction = row['prediction']  
    
    sari_score_easse = corpus_sari(
        orig_sents=[r_content],
        sys_sents=[prediction],
        refs_sents=[[s_content]]
        #refs_sents=[[simp] for simp in s_content['simplifications']]
    )
    
    sari_scores_easse.append(sari_score_easse)

  0%|          | 0/116 [00:00<?, ?it/s]

KeyError: 'label_text'

In [106]:
import numpy as np
print("Average SARI score:", np.mean(sari_scores_easse))

Average SARI score: 36.46781896155372


# Operation scores (add, keep, delete)

In [4]:
from easse.sari import get_corpus_sari_operation_scores
from tqdm.notebook import tqdm

add_scores = []
keep_scores = []
del_scores = []

for index, row in tqdm(df_gen.iterrows(), total=len(df_gen)):
    r_content = row['source_text'] 
    s_content = row['label_text'] #dataset['test'][index]  
    prediction = row['prediction']  
    
    add_score, keep_score, del_score = get_corpus_sari_operation_scores(
        orig_sents=[r_content],
        sys_sents=[prediction],
        refs_sents=[[s_content]]
        #refs_sents=[[simp] for simp in s_content['simplifications']] 
    )
    
    add_scores.append(add_score)
    keep_scores.append(keep_score)
    del_scores.append(del_score)

  0%|          | 0/116 [00:00<?, ?it/s]

# BLEU-4 (EASSE package)

In [110]:
from tqdm.notebook import tqdm
from easse.bleu import corpus_bleu
import numpy as np

bleu_scores_easse = []

for index, row in tqdm(df_gen.iterrows(), total=len(df_gen)):
    s_content = row['elaboration_sentence'] 
    prediction = row['pred_elaboration'] # "prediction" for BART
    
    bleu_score_easse = corpus_bleu(
        sys_sents=[prediction],
        refs_sents=[[s_content]]
    )
    
    bleu_scores_easse.append(bleu_score_easse)

print(f"Average BLEU score: {np.mean(bleu_scores_easse):.3f}")

  0%|          | 0/116 [00:00<?, ?it/s]

Average BLEU score: 3.738


### BLEU-1 & BLEU-2 (nltk + tokenizer-13A)

## Corpus bleu

In [111]:
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from sacrebleu.tokenizers.tokenizer_13a import Tokenizer13a
from tqdm import tqdm

# 13a tokenizer
tokenizer = Tokenizer13a()
smoothing_function = SmoothingFunction().method1

all_refs = []
all_preds = []

# Tokenize and collect references and predictions
for index, row in tqdm(df_gen.iterrows(), total=len(df_gen)):
    ref = row['elaboration_sentence']
    prediction = row['pred_elaboration'] # "prediction" for BART

    # Tokenize
    tokenized_ref = tokenizer(ref).split()
    tokenized_pred = tokenizer(prediction).split()
    
    all_refs.append([tokenized_ref]) 
    all_preds.append(tokenized_pred)

bleu1_score = corpus_bleu(all_refs, all_preds, weights=(1.0, 0, 0, 0), smoothing_function=smoothing_function)  # 1-gram
bleu2_score = corpus_bleu(all_refs, all_preds, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing_function)  # 2-gram
bleu4_score = corpus_bleu(all_refs, all_preds, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing_function)  # 4-gram

print(f"Corpus BLEU-1: {bleu1_score*100:.3f}")
print(f"Corpus BLEU-2: {bleu2_score*100:.3f}")
print(f"Corpus BLEU-4: {bleu4_score*100:.3f}")

100%|██████████████████████████████████████| 116/116 [00:00<00:00, 16134.08it/s]

Corpus BLEU-1: 13.147
Corpus BLEU-2: 1.519
Corpus BLEU-4: 0.126


# Sentence bleu

In [45]:
from nltk.translate.bleu_score import sentence_bleu
from sacrebleu.tokenizers.tokenizer_13a import Tokenizer13a
from transformers import BartTokenizer

bleu_scores_1 = []
bleu_scores_2 = []
bleu_scores_4 = []

# 13a tokenizer
tokenizer = Tokenizer13a()
# bart tokenizer
#tokenizer_b = BartTokenizer.from_pretrained('facebook/bart-base',use_fast=False) 

smoothing_function = SmoothingFunction().method1

for index, row in tqdm(df_gen.iterrows(), total=len(df_gen)):
    ref = row['elaboration_sentence']
    prediction = row['pred_elaboration']

    # tokenize
    tokenized_ref = tokenizer(ref).split()
    tokenized_pred = tokenizer(prediction).split()
    #tokenized_ref = tokenizer_b(ref)["input_ids"]
    #tokenized_pred = tokenizer_b(prediction)["input_ids"]
        
    bleu_score_1 = sentence_bleu([tokenized_ref],tokenized_pred,weights=(1, 0, 0, 0),smoothing_function=smoothing_function) # 1-gram
    bleu_score_2 = sentence_bleu([tokenized_ref],tokenized_pred,weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing_function) # 2-gram
    bleu_score_4 = sentence_bleu([tokenized_ref],tokenized_pred,weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing_function) # 4-gram
    bleu_scores_1.append(bleu_score_1)
    bleu_scores_2.append(bleu_score_2)
    bleu_scores_4.append(bleu_score_4)

import numpy as np
print(f"Average BLEU-1 score: {np.mean(bleu_scores_1)*100:.3f}")
print(f"Average BLEU-2 score: {np.mean(bleu_scores_2)*100:.3f}")
print(f"Average BLEU-4 score: {np.mean(bleu_scores_4)*100:.3f}")

100%|███████████████████████████████████████| 116/116 [00:00<00:00, 4293.61it/s]

Average BLEU-1 score: 16.922
Average BLEU-2 score: 6.008
Average BLEU-4 score: 3.106


## Tokenization - sample

In [125]:
# sample = df_gen.iloc[0]
ref = sample["label_text"]
pred = sample["prediction"]
print(tokenizer(ref)["input_ids"])
print(tokenizer(pred)["input_ids"])

[0, 10787, 109, 45, 33, 5, 418, 7, 120, 5, 1058, 51, 240, 4, 2]
[0, 1213, 109, 45, 33, 615, 418, 7, 582, 13, 1564, 4, 2]


# Show results

In [7]:
df_results = pd.DataFrame({
    'elaboration_sentence': df_gen['elaboration_sentence'],
    'pred_elaboration': df_gen['pred_elaboration'],
    'bleu-1': bleu_scores_1,
    'bleu-2': bleu_scores_2,
})

df_results.head()

,elaboration_sentence,pred_elaboration,bleu-1,bleu-2
0,Many do not have the money to get the training...,They do not have the money to pay for college.,0.530570,0.470300
1,A gauge is a kind of measuring stick.,A river is a river in the United States.,0.400000,0.210819
2,It sits in the water.,A gauge is a device that tracks the flow of wa...,0.250000,0.150756
3,They raced against the setting sun to search t...,There were no reports of injuries.,0.039493,0.013490
4,Turkey is larger than the state of Texas.,It is one of the United States' closest allies.,0.400000,0.066667


# Save results

In [8]:
df_results.to_csv("../data/bleu_scores/bleu_scores_bart-ft-c2sp-masked.csv", index=False)